### Programming Assignment 3 - Part II
##### Submitted By: Saurabh Kumar (SC22B146)
#### Solution of the "Clinical Decision Making Problem [Puterman]" and study of optimal policies

**Assumtion:** H = 2, L = 2  
  
**Probabilities:**  
gamma(h'|h) : the probability that a patient in health state h deteriorates to state h  
delta(h) : the probability a patient in health state h dies from liver failure prior to the next decision epoch  
beta (l|h) : the probability that a patient in health state h is offered a liver of quality  
phi(h) : the probability a patient in health state h is not o!ered a liver prior to the next decision epoch  

**Given that:**  
gamma(1∣1) = 0.8, gamma(2∣1) = 0.1,  gamma(1∣2) = 0, gamma(2∣2) = 0.6  
beta(1∣1) = 0.2, beta(2∣1) = 0.5,  beta(1∣2) = 0.3, beta(2∣2) = 0.5  
R(1∣1) = 5, R(2∣1) = 2,  R(1∣2) = 3, R(2∣2) = 1  

**Write down what would be delta(1) and delta(2)?**  
For health state h = 1,  
gamma(1∣1)+gamma(2∣1)+delta(1)=1  
This gives, delta(1) = 0.1  

For health state h = 2,  
gamma(1∣2)+gamma(2∣2)+delta(2)=1  
This gives, delta(2) = 0.4  

**Write down what would be phi(1) and phi(2)?**  
For health state h = 1,  
beta(1∣1)+beta(2∣1)+phi(1)=1  
This gives, phi(1) = 0.3  

For health state h = 2,  
beta(1∣2)+beta(2∣2)+phi(2)=1  
This gives, phi(2) = 0.2  



**Discount Factor**  
For this problem, the discount factor is **1**. This is because the model has **absorbing states** - death (delta) and post-transplant (tau). Any policy will eventually lead to one of these states, ending the process. Unlike typical infinite-horizon problems that require a discount factor less than 1 to ensure rewards don't go to infinity, this problem's structure guarantees a finite total reward. Therefore, we can maximize the undiscounted life expectancy by setting the discount factor to 1.

**Transition Probabilities p(j|s,a)**  
The transition probability p(j|s, a) is the chance of moving from a current state s to a next state j by taking action a. 
   
**Action a0 (Wait/Reject):** If wait, the transition probability is the product of two independent events: the change in health and the status of a new liver offer.  
The probability of moving from s=(h,l) to j=(h', l') is gamma(h'|h) * betta(l'|h).  
The probability of dying is delta(h).


**Action a1 (Accept):** If accept a liver, the transition is deterministic.  
We move to the post-transplant state tau with a probability of 1.

In [8]:
import numpy as np

# Model Parameters
gamma = np.array([[0.8, 0.1], [0.0, 0.6]]) # Health deterioration probabilities
beta = np.array([[0.2, 0.5], [0.3, 0.5]]) # Liver offer probabilities
R_hl = np.array([[5, 3], [2, 1]]) # Post-transplant rewards
delta = np.array([0.1, 0.4]) # Death probabilities
phi = np.array([0.3, 0.2]) # No-offer probabilities

# MDP
state_map = {
    (1, 1): 0, (1, 2): 1, (1, 'phi1'): 2,
    (2, 1): 3, (2, 2): 4, (2, 'phi1'): 5,
    'delta1': 6, 'tau1': 7
}
inv_state_map = {v: k for k, v in state_map.items()}
num_states, num_actions = 8, 2
P = np.zeros((num_actions, num_states, num_states))
R = np.zeros((num_actions, num_states))

# Transition and Reward Matrices
for s_idx, state in inv_state_map.items():
    if state in ['delta1', 'tau1']:
        P[0, s_idx, s_idx] = 1.0
        continue
    h, l = state
    # Action 0: wait/reject
    R[0, s_idx] = 1.0 - delta[h-1]
    for h_prime in range(h, 3):
        prob_h = gamma[h-1, h_prime-1]
        for l_prime in range(1, 3):
            P[0, s_idx, state_map[(h_prime, l_prime)]] += prob_h * beta[h_prime-1, l_prime-1]
        P[0, s_idx, state_map[(h_prime, 'phi1')]] += prob_h * phi[h_prime-1]
    P[0, s_idx, state_map['delta1']] = delta[h-1]
    # Action 1: accept
    if l != 'phi1':
        R[1, s_idx] = R_hl[h-1, l-1]
        P[1, s_idx, state_map['tau1']] = 1.0

# Value Iteration
V = np.zeros(num_states)
for i in range(100):
    V_old = V.copy()
    Q = R + 1.0 * (P @ V_old)
    invalid_states = [2, 5, 6, 7]
    Q[1, invalid_states] = -np.inf
    V = np.max(Q, axis=0)
    if np.max(np.abs(V - V_old)) < 1e-6:
        break

# Optimal Policy
policy = np.argmax(Q, axis=0)
action_map = {0: 'Wait/Reject', 1: 'Accept'}
print("Optimal Policy:")
for s_idx, state in inv_state_map.items():
    if isinstance(state, tuple) and state[1] != 'phi1':
        action = action_map[policy[s_idx]]
        print(f"State (Health={state[0]}, Liver={state[1]}): {action}")

Optimal Policy:
State (Health=1, Liver=1): Wait/Reject
State (Health=1, Liver=2): Wait/Reject
State (Health=2, Liver=1): Accept
State (Health=2, Liver=2): Wait/Reject


**Optimal Policy**
The optimal policy dictates the best action to take in each state to maximize the expected total life expectancy. The calculated optimal policy is:  
State (Health=1, Liver Quality=1): Wait/Reject  
State (Health=1, Liver Quality=2): Wait/Reject  
State (Health=2, Liver Quality=1): Accept  
State (Health=2, Liver Quality=2): Wait/Reject  

 
This policy shows that a healthy patient should wait for a high-quality organ, but an unhealthy patient should accept any available organ because the risk of waiting is too high.